# Introduction

In [1]:
from uarray import *
import pprint

## Creating Arrays

Everything (Operations and Symbols) are one of two types:

* Arrays: Reduce down to Array(length: Accessor, content: Accessor)
* Accessors: Accessors support calling Get(index: Accessor, content: Accessor) -> Array

### Scalar

We can add custom values as scalars:

In [2]:
a_scalar = scalar(123)
print(a_scalar)
a_scalar

123


Scalar(Value('123'))

We see that it's an Array with a length of `NoLengthAccessor()` and a content of `ScalarAccessor(123)`.

Now we can take the shape of a scalar:

In [3]:
a_scalar_shape = Shape(a_scalar)
a_scalar_shape

Shape(Scalar(Value('123')))

We see that the shape is not computed.

It just contains a reference to the arrays it operates on:

In [4]:
a_scalar_shape.operands

[Scalar(Value('123'))]

It also has a shorter representation we can print based on the Mathematics of Arrays symbols:

In [5]:
print(a_scalar_shape)

ρ(123)


We call `replace` to keep replacing expressions until no more are matched:

In [6]:
print(replace(a_scalar_shape))

Sequence(0, <>)


### Vector

We see that the shape of a scalar is `VectorAccessor`. We can create arrays with vector accessors with the `vector` helper functions.

In [7]:
??vector

Signature: vector(*values)
Docstring: <no docstring>
Source:   
def vector(*values):
    return vector_of(*(Value(v) for v in values))
File:      ~/p/uarray/uarray/uarray/core.py
Type:      function


In [8]:
a_vector = vector(1, 2, 3)
print(a_vector)
a_vector

Sequence(3, <1 2 3>)


Sequence(Value('3'), VectorCallable(Value('1'), Value('2'), Value('3')))

We can see the shape of a vector:

In [9]:
for x in Shape(a_vector).r:
    print(x)

Sequence((1 + ExtractLength(ρ(Call(<1 2 3>, _)))), <3, *Content(ρ(Call(<1 2 3>, _)))>)
Sequence((1 + ExtractLength(ρ(VectorIndexed(_, 1, 2, 3)))), <3, *Content(ρ(Call(<1 2 3>, _)))>)
Sequence((1 + ExtractLength(Sequence(0, <>))), <3, *Content(ρ(Call(<1 2 3>, _)))>)
Sequence((1 + 0), <3, *Content(ρ(Call(<1 2 3>, _)))>)
Sequence(1, <3, *Content(ρ(Call(<1 2 3>, _)))>)
Sequence(1, <3, *Content(ρ(VectorIndexed(_, 1, 2, 3)))>)
Sequence(1, <3, *Content(Sequence(0, <>))>)
Sequence(1, <3, *<>>)
Sequence(1, <3>)


Which is itself a vector. 

In [33]:
function(1, lambda i: Index(i, i))

Function(Index(Unbound('', variable_name=i37), Unbound('', variable_name=i37)), Unbound('', variable_name=i37))

### Iota

Now let's do something more interesting and create a range of values:

In [10]:
a_range = Iota(scalar(10))
print(a_range)

ι(10)


In [11]:
print(replace(Shape(a_range)))

Sequence(1, <10>)


This is a vector of length 10. Let's see that the 5th element is the integer 5.

In [12]:
# pprint.pprint(list(Index(vector(5), a_range).r))
print(replace(Index(vector(5), a_range)))

5


Here we use a new command `Index` which takes in a vector of indices and the an array and returns a subarray.

Let's try adding taking it's sum:

In [13]:
replace(ReduceVector(function(2, Add), Value(0), a_range))

Value('45')

In [14]:
sum(range(10))

45

Nowe we have an example taking the outer product of two iotas and indexing them

In [15]:
replaced = replace(OuterProduct(function(2, Multiply), Iota(scalar(4)), Iota(scalar(4))))
pprint.pprint(replaced)

Sequence(Value('4'),
         Function(Sequence(Value('4'),
                           Function(Scalar(Multiply(Unbound('', variable_name=i8),
                                                    Unbound('', variable_name=i10))),
                                    Unbound('', variable_name=i10))),
                  Unbound('', variable_name=i8)))


In [16]:
pprint.pprint(replace(Index(vector(2, 3), replaced)))

Scalar(Value('6'))


In [17]:
replace(BinaryOperation(function(2, Multiply), Iota(scalar(10)), scalar(5)))

Sequence(Value('10'), Function(Scalar(Multiply(Unbound('', variable_name=i14), Value('5'))), Unbound('', variable_name=i14)))

## Unbound Variables

### 0D

In [18]:
Z = UnboundWithDimension(0, "Z")
print(Z)
replaced_Z = replace(Z)
pprint.pprint(replaced_Z)

Z^0
Scalar(Content(Unbound('', variable_name=Z)))


We can then replace that later with the actual value, after it has been simplified:

In [19]:
replace(matchpy.substitute(replaced_Z, {"Z": scalar(10)}))

Scalar(Value('10'))

### 1D

This is all fine and good, but what about an array with more dimensions, like a 1D array?

In [20]:
A = UnboundWithDimension(1, "A")
print(A)
replaced_A = replace(A)
pprint.pprint(replaced_A)

A^1
Sequence(ExtractLength(Unbound('', variable_name=A)),
         Function(Scalar(Content(Call(Content(Unbound('', variable_name=A)),
                                      Unbound('', variable_name=i15)))),
                  Unbound('', variable_name=i15)))


We can get the shape of this:

In [21]:
replaced_shape_a = replace(Shape(replaced_A))
print(replaced_shape_a)

Sequence(1, <ExtractLength(A)>)


Now let's get an item from this:

In [22]:
replaced_get_A = replace(Index(vector(10), A))
print(replaced_get_A)

Content(Call(Content(A), 10))


Now let's substitute and see if it is correct

In [23]:
print(replace(matchpy.substitute(replaced_A, {"A": vector(0, 1, 2)})))

Sequence(3, (i15 -> VectorIndexed(i15, 0, 1, 2)))


### 2D

Now let's make a matrix


In [24]:
B = UnboundWithDimension(2, "B")


In [25]:
pprint.pprint(replace(B))

Sequence(ExtractLength(Unbound('', variable_name=B)),
         Function(Sequence(ExtractLength(Call(Content(Unbound('', variable_name=B)),
                                              Unbound('', variable_name=i17))),
                           Function(Scalar(Content(Call(Content(Call(Content(Unbound('', variable_name=B)),
                                                                     Unbound('', variable_name=i17))),
                                                        Unbound('', variable_name=i18)))),
                                    Unbound('', variable_name=i18))),
                  Unbound('', variable_name=i17)))


In [26]:
print(replace(Index(vector_of(Unbound("j"), Unbound("i")), replace(B))))

Content(Call(Content(Call(Content(B), j)), i))


# Paper Example

Now let's look at the example from the paper in `uarray-docs`.

In [27]:
A = UnboundWithDimension(3, "A")
B = UnboundWithDimension(3, "B")
r = InnerProduct(
    function(2, Add),
    function(2, Multiply),
    Index(vector(1, 0), A),
    Index(
        vector(2),
        OuterProduct(
            function(2, Multiply),
            Index(vector(1, 0), A),
            Index(vector(0, 1), B)
        )
    )
)
print(r)

((Sequence(2, <1 0>) ψ A^3) (i21, i22 -> (i21 + i22))·(i23, i24 -> (i23 * i24)) (Sequence(1, <2>) ψ ((Sequence(2, <1 0>) ψ A^3) ·(i25, i26 -> (i25 * i26)) (Sequence(2, <0 1>) ψ B^3))))


In [28]:
replaced_r = replace(r)
pprint.pprint(replaced_r)

BinaryOperation(Function(Multiply(Unbound('', variable_name=i25),
                                  Unbound('', variable_name=i26)),
                         Unbound('', variable_name=i25),
                         Unbound('', variable_name=i26)),
                Scalar(Content(Call(Content(Call(Content(Call(Content(Unbound('', variable_name=A)),
                                                              Value('1'))),
                                                 Value('0'))),
                                    Value('2')))),
                InnerProduct(Function(Add(Unbound('', variable_name=i21),
                                          Unbound('', variable_name=i22)),
                                      Unbound('', variable_name=i21),
                                      Unbound('', variable_name=i22)),
                             Function(Multiply(Unbound('', variable_name=i23),
                                               Unbound('', variable_name=i24)),
            

This is a bit opaque, but does represent the right optimized form. Let's now conver this to a string of NumPy code:

In [29]:
import numpy as np

In [30]:
A = np.arange(30).reshape((2, 3, 5))
B = 2 + A

In [31]:
res = replace(matchpy.substitute(replaced_r, {"A": Numpy("A"), "B": Numpy("B")}))
pprint.pprint(res)

BinaryOperation(Function(Multiply(Unbound('', variable_name=i25),
                                  Unbound('', variable_name=i26)),
                         Unbound('', variable_name=i25),
                         Unbound('', variable_name=i26)),
                Numpy(('A', (1, 0, 2))),
                InnerProduct(Function(Add(Unbound('', variable_name=i21),
                                          Unbound('', variable_name=i22)),
                                      Unbound('', variable_name=i21),
                                      Unbound('', variable_name=i22)),
                             Function(Multiply(Unbound('', variable_name=i23),
                                               Unbound('', variable_name=i24)),
                                      (Unbound('', variable_name=i23),
                                       Unbound('', variable_name=i24))),
                             Sequence(ExtractLength(Numpy(('A', (1, 0)))),
                                      Funct

In [32]:
eval(res.code)

AttributeError: 'BinaryOperation' object has no attribute 'code'

In [ ]:
replacer.matcher.as_graph()